In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from sklearn.metrics import mean_squared_error

In [2]:
# Load the log dataset (replace with your dataset file)
data = pd.read_csv('../dataset/log_data.csv')

In [3]:
# Convert timestamp to datetime objects
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [4]:
# Calculate time difference in seconds from the minimum timestamp
min_timestamp = data['timestamp'].min()
data['time_diff'] = (data['timestamp'] - min_timestamp).dt.total_seconds()

In [5]:
# Select relevant features for anomaly detection
# In this case, we'll use time_diff, activity, and username
features = data[['time_diff', 'activity', 'username']]


In [6]:
# Convert categorical features to numerical using label encoding
label_encoders = {}
for col in ['activity', 'username']:
    label_encoders[col] = LabelEncoder()
    features[col] = label_encoders[col].fit_transform(features[col])

C:\Users\arunh\AppData\Local\Temp\ipykernel_1860\1446478807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[col] = label_encoders[col].fit_transform(features[col])
C:\Users\arunh\AppData\Local\Temp\ipykernel_1860\1446478807.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[col] = label_encoders[col].fit_transform(features[col])


In [7]:
# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [8]:
# Split the data into train and test sets
X_train, X_test = train_test_split(scaled_features, test_size=0.2, random_state=42)


In [9]:
# ... Build the autoencoder model ...
input_dim = X_train.shape[1]
encoding_dim = 10  # Adjust based on your data complexity
autoencoder = Sequential()
autoencoder.add(Dense(encoding_dim, input_shape=(input_dim,), activation='relu'))
autoencoder.add(Dense(input_dim, activation='linear'))
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [10]:
# ... Train the autoencoder model ...
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_data=(X_test, X_test))


Epoch 1/50
250/250 [==============================] - 1s 2ms/step - loss: 0.6661 - val_loss: 0.2783
Epoch 2/50
250/250 [==============================] - 0s 2ms/step - loss: 0.1315 - val_loss: 0.0571
Epoch 3/50
250/250 [==============================] - 0s 2ms/step - loss: 0.0366 - val_loss: 0.0251
Epoch 4/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0204 - val_loss: 0.0170
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0156 - val_loss: 0.0140
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0132 - val_loss: 0.0121
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0114 - val_loss: 0.0104
Epoch 8/50
250/250 [==============================] - 0s 2ms/step - loss: 0.0097 - val_loss: 0.0088
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0080 - val_loss: 0.0071
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0063 - val_loss: 0.0054

In [11]:
# Reconstruct the data using the trained autoencoder
reconstructed_data = autoencoder.predict(scaled_features)
mse = np.mean(np.power(scaled_features - reconstructed_data, 2), axis=1)

313/313 [==============================] - 0s 1ms/step


In [12]:
# Define a threshold to identify anomalies
threshold = np.percentile(mse, 95)  # Adjust based on your data

In [13]:
# Identify anomalies
anomalies = data[mse > threshold]

In [14]:
# Define actionable insights templates
insight_templates = {
    'Unauthorized Access Detected': {
        'severity': 'High',
        'rule': 'Access Control Enforcement',
        'template': "Anomaly Details:\n- Timestamp: {timestamp}\n- User: {username}\n- Activity: {activity}\n- Recommendation: Investigate the unauthorized access by user '{username}' at the given timestamp. Check for signs of compromise and take appropriate action to secure the account and prevent further breaches."
    },
    # Define more templates for other compliance rules
}

In [15]:
# Generate actionable insights
actionable_insights = []
for _, anomaly_row in anomalies.iterrows():
    insight_template = insight_templates.get('Unauthorized Access Detected')  # Replace with relevant template
    if insight_template:
        actionable_insights.append(insight_template['template'].format(
            timestamp=anomaly_row['timestamp'],
            username=anomaly_row['username'],
            activity=anomaly_row['activity']
        ))

In [16]:
# Print actionable insights
print("Actionable Insights:")
for insight in actionable_insights:
    print(insight)


Actionable Insights:
Anomaly Details:
- Timestamp: 2023-08-06 14:37:52
- User: qgray
- Activity: AccessDenied
- Recommendation: Investigate the unauthorized access by user 'qgray' at the given timestamp. Check for signs of compromise and take appropriate action to secure the account and prevent further breaches.
Anomaly Details:
- Timestamp: 2023-08-05 07:21:57
- User: steven76
- Activity: AccessDenied
- Recommendation: Investigate the unauthorized access by user 'steven76' at the given timestamp. Check for signs of compromise and take appropriate action to secure the account and prevent further breaches.
Anomaly Details:
- Timestamp: 2023-06-21 08:43:08
- User: howardtiffany
- Activity: AccessDenied
- Recommendation: Investigate the unauthorized access by user 'howardtiffany' at the given timestamp. Check for signs of compromise and take appropriate action to secure the account and prevent further breaches.
Anomaly Details:
- Timestamp: 2023-08-07 09:30:27
- User: rsanchez
- Activity: